In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [20]:
spark_df = spark.read.format("csv").option("header", "true").load("challenge.csv")

In [3]:
spark_df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [21]:
from pyspark.sql.functions import *
spark_df = spark_df.withColumn("Is_Mexico", when(spark_df.Country == "Mexico", 'YES').otherwise("NO"))

In [22]:
#spark_df.show()
#spark_df = spark_df.withColumn('Bytes_used', spark_df.Bytes_used.cast('float'))
spark_df = spark_df.withColumn("Bytes_used", spark_df.Bytes_used.cast('float'))
#spark_df.printSchema()

In [23]:
import pyspark.sql.functions as sqlfunc
new_df = spark_df.groupBy("Is_Mexico").agg(sqlfunc.sum('Bytes_used'))
new_df.show()

+---------+---------------+
|Is_Mexico|sum(Bytes_used)|
+---------+---------------+
|      YES|         6293.0|
|       NO|       508076.0|
+---------+---------------+



In [33]:
new_df = spark_df.groupBy("Country").agg(sqlfunc.countDistinct(spark_df.ip_address).alias("no_of_ips"))

In [39]:
new_df.sort(col('no_of_ips').desc()).show()

+--------------+---------+
|       Country|no_of_ips|
+--------------+---------+
|         China|      172|
|     Indonesia|      114|
|   Philippines|       65|
|        Russia|       56|
|        Brazil|       35|
|        Poland|       31|
|        Sweden|       28|
|         Japan|       25|
|Czech Republic|       23|
|      Portugal|       23|
|        France|       21|
|          Peru|       19|
|      Colombia|       17|
| United States|       15|
|       Ukraine|       14|
|     Argentina|       14|
|        Mexico|       13|
|      Thailand|       12|
|       Nigeria|       11|
|        Canada|       11|
+--------------+---------+
only showing top 20 rows

